In [17]:
import numpy as np
import torch
import pandas as pd
import optuna
import torch.optim as optim
import matplotlib.pyplot as plt
import math

from sklearn import model_selection
from NMFAE_init import NMFAE, train_NMFAE
from functools import partial 
from skopt import space
from skopt import gp_minimize
from functions import simulate_counts
from sklearn.decomposition import NMF
from optuna.trial import TrialState

In [18]:
def MSE_AE(X, nsig, lr, batch_size):
    model = NMFAE(nsig)
    kf = model_selection.KFold()

    out_err = []
    loss_function = torch.nn.MSELoss(reduction='mean')
    optimizer = torch.optim.SGD(model.parameters(), lr = lr)
    #optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)


    for train, test in kf.split(X):
        x_train = pd.DataFrame(X).iloc[train,:]
        x_test = pd.DataFrame(X).iloc[test,:]

        train_NMFAE(
            epochs = 200, 
            model = model, 
            x_train = x_train, 
            loss_function = loss_function, 
            optimizer = optimizer,
            batch_size = int(batch_size)
            )

        cv_test_tensor = torch.tensor(x_test.values, 
                                        dtype = torch.float32)

        cv_fit = model(cv_test_tensor)
        out_err.append(float(loss_function(cv_fit,cv_test_tensor).detach().numpy()))
    print(out_err)
    return np.mean(out_err)

In [19]:
def skopt_AE(X):
    def optimize(params, param_names, X):
        params = dict(zip(param_names, params))
        lr = params['lr'] 
        nsig = params['nsig']
        #optimizer_name = params['optimizer_name']
        batch_size = params['batch_size']
        res = MSE_AE(X, nsig, lr, batch_size)
        print(res)
        return res

    param_space = [
        space.Integer(2, 15, name = "nsig"),
        space.Real(0.000001, 1, prior = "uniform", name = "lr"),
        space.Categorical([8, 16, 32, 64], name = 'batch_size'),
        #space.Categorical( ["SGD"], name = "optimizer_name")
        #space.Categorical( ["Adam", "RMSprop"], name = "optimizer_name")
    ]

    param_names = [
        "nsig", 
        "lr",
        "batch_size",
        "optimizer_name"
    ]

    optimization_AE = partial(
        optimize,
        param_names = param_names,
        X = X
    )

    resAE = gp_minimize(
        optimization_AE, 
        dimensions = param_space,
        n_calls = 10,
        n_random_starts = 10,
        verbose = 0,
        n_jobs=3)
    
    return resAE.x[0]

In [20]:
def rank_analysis(npatients, nsigs):
  X, _,_ = simulate_counts(nsigs, npatients)
  X = X.transpose()
  return(skopt_AE(X))
asd = [rank_analysis(200,3) for _ in range(5)]
print(asd)

[nan, nan, nan, nan, nan]
nan
[nan, nan, nan, nan, nan]
nan
[nan, nan, nan, nan, nan]
nan
[nan, nan, nan, nan, nan]
nan


KeyboardInterrupt: 